In [ ]:
!pip install ultralytics  # This includes YOLOv8
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define paths
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os
from IPython.display import display, Image
import matplotlib.pyplot as plt
# dir_path = "/content/drive/MyDrive/ObjectDetection-Tasks" - - - - - ORIGINAL
# filepath = os.path.join(dir_path, "Task-for-model-2")
# dataset_path = os.path.join(filepath, 'retrainDataset/combinedRetrainData')
# dir = os.path.join(dataset_path, "yoloSamLabelData")


dir_path = "/content/drive/MyDrive/ObjectDetection-Tasks"
filepath = os.path.join(dir_path, "APD-Detection")
dataset_path = os.path.join(filepath, 'dataset0')
dir = os.path.join(dataset_path, 'data0/yoloSamLabel')


labels_source = os.path.join(dataset_path, "labelData")
images_source = os.path.join(dataset_path,  "image")

### Using custom build syntax

In [ ]:
# def visualize_yolo_annotation(image_path, annotation_path, class_names):
#     """
#     Visualizes YOLO annotations on an image.

#     Args:
#         image_path (str): Path to the image file.
#         annotation_path (str): Path to the YOLO annotation file (.txt).
#         class_names (list): A list of class names, indexed by class_id.
#     """
#     # Load the image
#     image = cv2.imread(image_path)
#     if image is None:
#         print(f"Error: Could not load image from {image_path}")
#         return None

#     # Convert BGR to RGB for proper color display
#     image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     h, w, _ = image_rgb.shape

#     # Check if annotation file exists
#     if not os.path.exists(annotation_path):
#         print(f"Error: Annotation file not found at {annotation_path}")
#         return None

#     # Read the annotation file
#     try:
#         with open(annotation_path, 'r') as f:  # Fixed: 'txt' -> 'r'
#             annotations = f.readlines()
#     except Exception as e:
#         print(f"Error reading annotation file: {e}")
#         return None

#     # Process each annotation
#     for annotation in annotations:
#         parts = annotation.strip().split()
#         if len(parts) != 5:
#             print(f"Warning: Invalid annotation format in line: {annotation.strip()}")
#             continue

#         try:
#             class_id = int(parts[0])
#             center_x_norm, center_y_norm, width_norm, height_norm = map(float, parts[1:])

#             # Convert normalized coordinates to absolute pixel coordinates
#             center_x = int(center_x_norm * w)
#             center_y = int(center_y_norm * h)
#             box_width = int(width_norm * w)
#             box_height = int(height_norm * h)

#             # Calculate top-left and bottom-right corner coordinates
#             x1 = int(center_x - box_width / 2)
#             y1 = int(center_y - box_height / 2)
#             x2 = int(center_x + box_width / 2)
#             y2 = int(center_y + box_height / 2)

#             # Ensure coordinates are within image boundaries
#             x1 = max(0, min(x1, w))
#             y1 = max(0, min(y1, h))
#             x2 = max(0, min(x2, w))
#             y2 = max(0, min(y2, h))

#             # Draw the bounding box
#             color = (0, 255, 0)  # Green color for bounding box
#             thickness = 2
#             cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, thickness)

#             # Display the class name
#             if class_id < len(class_names):
#                 label = class_names[class_id]
#                 font = cv2.FONT_HERSHEY_SIMPLEX
#                 font_scale = 0.7
#                 font_thickness = 2

#                 # Get text size for background
#                 text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]

#                 # Draw background rectangle for text
#                 cv2.rectangle(image_rgb, (x1, y1 - text_size[1] - 10),
#                              (x1 + text_size[0], y1), color, -1)

#                 # Put text
#                 cv2.putText(image_rgb, label, (x1, y1 - 5),
#                            font, font_scale, (0, 0, 0), font_thickness, cv2.LINE_AA)

#         except Exception as e:
#             print(f"Error processing annotation: {e}")
#             continue

#     return image_rgb

# def display_image(image):
#     """Display image using matplotlib (better for Colab)"""
#     plt.figure(figsize=(12, 8))
#     plt.imshow(image)
#     plt.axis('off')
#     plt.tight_layout()
#     plt.show()

In [ ]:
# # Example usage for Colab:
# if __name__ == "__main__":
#     # Mount Google Drive (if files are in Drive)
#     # from google.colab import drive
#     # drive.mount('/content/drive')

#     # Example paths - UPDATE THESE WITH YOUR ACTUAL PATHS
#     image_file = images_source  # Replace with your actual image path
#     annotation_file = dir  # Replace with your actual annotation path

#     # Alternative: Upload files directly to Colab
#     # from google.colab import files
#     # uploaded = files.upload()
#     # image_file = list(uploaded.keys())[0]  # Use first uploaded file as image
#     annotation_file = image_file.replace('.jpg', '.txt').replace('.png', '.txt')

#     # Define your class names in the correct order
#     my_class_names = ["fork", "rice", "menu", "innerBox","outerBox"]

#     # Visualize the annotations
#     result_image = visualize_yolo_annotation(image_file, annotation_file, my_class_names)

#     if result_image is not None:
#         display_image(result_image)
#         print("Annotation visualization completed successfully!")
#     else:
#         print("Failed to visualize annotations.")

### Using ultralytics utilities to display annotated image

In [ ]:
# Function utilizing ultralytics to display segmentation of images
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import math

def visualize_yolo_segmentation(image_path, annotation_path, class_names):
    """
    Visualizes YOLO segmentation annotations on an image.
    """
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image from {image_path}")
        return None

    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image_rgb.shape

    # Check if annotation file exists
    if not os.path.exists(annotation_path):
        print(f"Error: Annotation file not found at {annotation_path}")
        return None

    # Define colors for different classes
    colors = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
        (255, 0, 255), (0, 255, 255), (128, 0, 0), (0, 128, 0),
        (128, 128, 0), (128, 0, 128), (0, 128, 128), (128, 128, 128)
    ]

    # Read and process annotations
    try:
        with open(annotation_path, 'r') as f:
            annotations = f.readlines()

        for i, annotation in enumerate(annotations):
            parts = annotation.strip().split()

            if not parts:
                continue

            class_id = int(parts[0])
            polygon_coords = list(map(float, parts[1:]))

            # Convert normalized coordinates to pixel coordinates
            pixel_coords = []
            for j in range(0, len(polygon_coords), 2):
                if j + 1 < len(polygon_coords):
                    x = int(polygon_coords[j] * w)
                    y = int(polygon_coords[j + 1] * h)
                    pixel_coords.append([x, y])

            if len(pixel_coords) < 3:
                continue

            polygon = np.array(pixel_coords, dtype=np.int32)
            color = colors[class_id % len(colors)]

            # Draw filled polygon with transparency
            mask = np.zeros_like(image_rgb)
            cv2.fillPoly(mask, [polygon], color)
            alpha = 0.3
            image_rgb = cv2.addWeighted(image_rgb, 1, mask, alpha, 0)

            # Draw polygon outline
            cv2.polylines(image_rgb, [polygon], True, color, 2)

            # Add class label
            if class_id < len(class_names):
                label = class_names[class_id]
                M = cv2.moments(polygon)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])

                    font = cv2.FONT_HERSHEY_SIMPLEX
                    font_scale = 0.7
                    font_thickness = 2

                    text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]
                    cv2.rectangle(image_rgb,
                                (cx - text_size[0]//2 - 5, cy - text_size[1] - 5),
                                (cx + text_size[0]//2 + 5, cy + 5),
                                color, -1)
                    cv2.putText(image_rgb, label,
                              (cx - text_size[0]//2, cy),
                              font, font_scale, (255, 255, 255), font_thickness, cv2.LINE_AA)

        return image_rgb

    except Exception as e:
        print(f"Error processing annotations: {e}")
        return None

def calculate_grid_size(num_images, max_cols=4):
    """
    Calculate optimal grid dimensions for subplots.

    Args:
        num_images (int): Number of images to display
        max_cols (int): Maximum number of columns in the grid

    Returns:
        tuple: (rows, cols) for subplot grid
    """
    cols = min(num_images, max_cols)
    rows = math.ceil(num_images / cols)
    return rows, cols

def visualize_segmentation_batch(images_dir, labels_dir, class_names, num_images=4, max_cols=4, figsize=(20, 15)):
    """
    Visualize multiple segmentation images in an adaptive grid.

    Args:
        images_dir (str): Directory containing images
        labels_dir (str): Directory containing annotation files
        class_names (list): List of class names
        num_images (int): Number of images to visualize (use None for all images)
        max_cols (int): Maximum number of columns in the grid
        figsize (tuple): Figure size (width, height)
    """
    # Get all image files
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Limit number of images if specified
    if num_images is not None:
        image_files = image_files[:num_images]

    if not image_files:
        print("No image files found in the directory.")
        return

    num_images = len(image_files)
    rows, cols = calculate_grid_size(num_images, max_cols)

    # Create subplot grid
    fig, axes = plt.subplots(rows, cols, figsize=figsize)

    # If only one image, convert axes to array for consistent indexing
    if num_images == 1:
        axes = np.array([axes])
    if rows == 1:
        axes = axes.reshape(1, -1)
    if cols == 1:
        axes = axes.reshape(-1, 1)

    # Flatten axes array for easy iteration
    axes_flat = axes.flatten()

    processed_count = 0
    for i, img_file in enumerate(image_files):
        if processed_count >= len(axes_flat):
            break

        image_path = os.path.join(images_dir, img_file)
        annotation_path = os.path.join(labels_dir, os.path.splitext(img_file)[0] + '.txt')

        if os.path.exists(annotation_path):
            result_image = visualize_yolo_segmentation(image_path, annotation_path, class_names)
            if result_image is not None:
                axes_flat[processed_count].imshow(result_image)
                axes_flat[processed_count].set_title(f"{img_file}", fontsize=10)
                axes_flat[processed_count].axis('off')
                processed_count += 1
            else:
                print(f"Failed to process: {img_file}")
        else:
            print(f"Annotation not found for: {img_file}")

    # Hide any unused subplots
    for j in range(processed_count, len(axes_flat)):
        axes_flat[j].axis('off')

    plt.tight_layout()
    plt.show()

    print(f"Successfully processed {processed_count} out of {num_images} images")

def visualize_single_segmentation(image_path, annotation_path, class_names, figsize=(12, 8)):
    """
    Visualize a single segmentation image.

    Args:
        image_path (str): Path to the image file
        annotation_path (str): Path to the annotation file
        class_names (list): List of class names
        figsize (tuple): Figure size
    """
    result_image = visualize_yolo_segmentation(image_path, annotation_path, class_names)

    if result_image is not None:
        plt.figure(figsize=figsize)
        plt.imshow(result_image)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    else:
        print("Failed to visualize segmentation.")

### History

In [ ]:
# # Syntax to utilize the function above

# # Define your directories
# images_dir = images_source
# labels_dir = dir

# # # Define your class names
# my_class_names = ["fork", "rice", "menu", "innerBox", "outerBox"]

# # Get a list of image files
# image_extensions = ('.jpg', '.jpeg', '.png')
# image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(image_extensions)]

# # Process the first image
# if image_files:
#     first_image_name = image_files[0]
#     image_file = os.path.join(images_dir, first_image_name)
#     annotation_file = os.path.join(labels_dir, os.path.splitext(first_image_name)[0] + '.txt')

#     print(f"Processing: {first_image_name}")
#     print(f"Image path: {image_file}")
#     print(f"Annotation path: {annotation_file}")

#     if os.path.exists(annotation_file):
#         # Visualize segmentation
#         result_image = visualize_yolo_segmentation(image_file, annotation_file, my_class_names)

#         if result_image is not None:
#             display_segmentation_result(result_image, f"Segmentation: {first_image_name}")

#             # Also show annotation file content for verification
#             print("\nAnnotation file content:")
#             with open(annotation_file, 'r') as f:
#                 for i, line in enumerate(f):
#                     parts = line.strip().split()
#                     if parts:
#                         class_id = int(parts[0])
#                         num_points = (len(parts) - 1) // 2
#                         print(f"Line {i}: Class {class_id} ({my_class_names[class_id]}), Points: {num_points}")
#         else:
#             print("Failed to visualize segmentation.")
#     else:
#         print(f"Annotation file not found: {annotation_file}")
# else:
#     print("No image files found in the directory.")

In [ ]:
# Function to display multiple images
# def visualize_multiple_segmentations(images_dir, labels_dir, class_names, num_images=4):
#     """Visualize multiple segmentation images"""
#     image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

#     fig, axes = plt.subplots(2, 2, figsize=(20, 15))
#     axes = axes.ravel()

#     processed = 0
#     for i, img_file in enumerate(image_files):
#         if processed >= num_images:
#             break

#         image_path = os.path.join(images_dir, img_file)
#         annotation_path = os.path.join(labels_dir, os.path.splitext(img_file)[0] + '.txt')

#         if os.path.exists(annotation_path):
#             result_image = visualize_yolo_segmentation(image_path, annotation_path, class_names)
#             if result_image is not None:
#                 axes[processed].imshow(result_image)
#                 axes[processed].set_title(f"Segmentation: {img_file}", fontsize=12)
#                 axes[processed].axis('off')
#                 processed += 1

#     # Hide unused subplots
#     for j in range(processed, len(axes)):
#         axes[j].axis('off')

#     plt.tight_layout()
#     plt.show()

# # Usage for multiple images
# # Visualize exactly 10 images (will show error if less than 10 available)
# visualize_segmentation_batch(images_dir, labels_dir, my_class_names, num_images=10, max_cols=4)

# # Visualize ALL images in the directory
# visualize_segmentation_batch(images_dir, labels_dir, my_class_names, num_images=None, max_cols=3)

# # Different grid configurations
# visualize_segmentation_batch(images_dir, labels_dir, my_class_names, num_images=8, max_cols=2)  # 4x2 grid
# visualize_segmentation_batch(images_dir, labels_dir, my_class_names, num_images=12, max_cols=6)  # 2x6 grid

# Visualize just one specific image
# image_file = os.path.join(images_dir, "your_specific_image.jpg")
# annotation_file = os.path.join(labels_dir, "your_specific_image.txt")
# visualize_single_segmentation(image_file, annotation_file, my_class_names)


In [ ]:
# # Check the problematic annotation file
# annotation_file = os.path.join(labels_dir, os.path.splitext(first_image_name)[0] + '.txt')

# print(f"Contents of {annotation_file}:")
# with open(annotation_file, 'r') as f:
#     lines = f.readlines()
#     for i, line in enumerate(lines):
#         parts = line.strip().split()
#         print(f"Line {i}: {parts} (Number of values: {len(parts)})")

In [ ]:
# # Auto visualize in these group with error labelling

# # # Define your directories
# images_dir = images_source
# labels_dir = dir

# # # Define your class names
# my_class_names = ["fork", "rice", "menu", "innerBox", "outerBox"]

# def smart_visualization(images_dir, labels_dir, class_names, desired_count=10):
#     """
#     Smart visualization that handles cases where fewer images are available.
#     """
#     image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

#     available_count = len(image_files)
#     actual_count = min(desired_count, available_count)

#     print(f"Requested: {desired_count} images")
#     print(f"Available: {available_count} images")
#     print(f"Will display: {actual_count} images")

#     if actual_count == 0:
#         print("No images to display.")
#         return

#     visualize_segmentation_batch(images_dir, labels_dir, class_names, num_images=actual_count)

# # Usage - automatically adapts to available images
# smart_visualization(images_dir, labels_dir, my_class_names, desired_count=20)

### Display batched images from auto_annotate

In [ ]:
# This function to display image from auto_annotate function in other notebook, with batched processing, where the amount of images is more than 200

# # Define your directories
images_dir = images_source
labels_dir = dir

# # Define your class names
my_class_names = ["person_with_helmet_forklift", "person_with_mask_forklift","person_with_mask_helmet_forklift","person_with_mask_nonForklift","person_without_mask_helmet_forklift","person_without_mask_nonForklift"]                              #["fork", "rice", "menu", "innerBox", "outerBox"]

from ultralytics import YOLO
import os

def smart_visualization_batched(images_dir, labels_dir, class_names, desired_count=20, batch_size=20):
    """
    Smart visualization that handles large numbers of images by processing them in batches.
    Correctly iterates through all images across multiple batches.
    """
    # Get all image files
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Limit to desired_count and ensure we don't exceed available images
    available_count = len(image_files)
    actual_count = min(desired_count, available_count)

    print(f"Requested: {desired_count} images")
    print(f"Available: {available_count} images")
    print(f"Will display: {actual_count} images in batches of {batch_size}")

    if actual_count == 0:
        print("No images to display.")
        return

    # Process and display images in batches
    for batch_start in range(0, actual_count, batch_size):
        batch_end = min(batch_start + batch_size, actual_count)
        current_batch_files = image_files[batch_start:batch_end]  # Correctly slice the next batch

        print(f"Displaying batch {batch_start//batch_size + 1}: images {batch_start} to {batch_end-1}")

        # Process the current batch
        for img_file in current_batch_files:
            image_path = os.path.join(images_dir, img_file)
            annotation_path = os.path.join(labels_dir, os.path.splitext(img_file)[0] + '.txt')

            if os.path.exists(annotation_path):
                # Use your existing visualization function for a single image
                visualize_single_segmentation(image_path, annotation_path, class_names)
            else:
                print(f"Annotation not found for: {img_file}")

# Load your model (assuming you have a model for visualization)
#model = YOLO("yolo11n.pt")  # Load your preferred model

# Usage - automatically adapts to available images
smart_visualization_batched(images_dir, labels_dir, my_class_names, desired_count=602, batch_size=30)